# Import package

In [28]:
#Import package
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pandas_datareader import data as web
import requests

# import yfinance as yf
# import lxml
# from bs4 import BeautifulSoup #BeutifulSoup

# Data Preparation

## Web Scrapping

This section will perform web scrapping to scrap all S&P500 tickers on wikipedia. https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We can simply use padnas to get the table from wiki.

In [30]:
#Store the S&P information in pandas dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
tickers_df_list = pd.read_html(wiki_url)
tickers_df = tickers_df_list[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [10]:
#Turn pandas dataframe to the list
tickers = tickers_df['Symbol'].values.tolist()
print(len(tickers))

503


## Download the financial data

In [32]:
#Download the financial data from Alpha vantage
function = 'TIME_SERIES_INTRADAY'
interval = '60min'
month = '2000-01'
datatype = 'json'
outputsize = 'compact'
alpha_vantage_apikey = 'RZ2BSD9KOG0RKCUL'
ticker = 'AAPL'

url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={alpha_vantage_apikey}'
r = requests.get(url)
r.
r.json()

{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2023-07-28 19:00:00',
  '4. Interval': '60min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (60min)': {'2023-07-28 19:00:00': {'1. open': '195.8500',
   '2. high': '195.9500',
   '3. low': '195.7800',
   '4. close': '195.9500',
   '5. volume': '20654'},
  '2023-07-28 18:00:00': {'1. open': '195.8000',
   '2. high': '195.8600',
   '3. low': '195.7200',
   '4. close': '195.8400',
   '5. volume': '22108'},
  '2023-07-28 17:00:00': {'1. open': '195.6800',
   '2. high': '195.8300',
   '3. low': '195.6200',
   '4. close': '195.7750',
   '5. volume': '359879'},
  '2023-07-28 16:00:00': {'1. open': '195.8400',
   '2. high': '195.8700',
   '3. low': '195.3000',
   '4. close': '195.6700',
   '5. volume': '15958267'},
  '2023-07-28 15:00:00': {'1. open': '195.9250',
   '2. high': '196.1500',
   '3. low': '195.4200',
   '4. c